## Calculating the Trajectory Errors

- how to calculate the trajectory errors and seperate the files into bad and good trajectories

- Currently need to work out how to get python to show the integration errors between the forward and reverse trajectories

In [1]:
# STEP 0: Must be done before anything each time notebook is loaded
# Import PySplit

import pysplit as py
import numpy as np
import pickle as pickle
from pprint import pprint
import os

#### Trajectory generation
- basic trajectory, test used for just this analysis

In [27]:
# STEP 1: TRAJECTORY GENERATION (not yet tested on mac)
# check out bulk_trajen_example.py

#directory setup
working_dir = r'/Volumes/Seagate_Backup/hysplit4/working'
storage_dir = r'/Volumes/Seagate_Backup/hysplit4/pysplitgen/test'
meteo_dir = r'/Volumes/Seagate_Backup/HYSPLIT/ncep_monYYYY'
# meteo_dir = r'C:\hysplit4\gdas'

#file basename
basename = 'test'

#Arguments
years = [2007]
#years=range(2006,2017)
#months = [1, 13]
months=range(1,13)
hours = [4]
altitudes = [1100]
location = (-10.25, 105.4)
runtime = -168

#Generate trajs
py.generate_bulktraj(basename, working_dir, storage_dir, meteo_dir,
                          years, months, hours, altitudes, location, runtime,
                          monthslice=slice(0, 32, 1), get_reverse=True, get_clipped=True, hysplit="/Volumes/Seagate_Backup/hysplit4/exec/hyts_std")

#### Grouping the trajectories

- determine the distance between the start and end of trajectories, the vector 
- can seperate the rainy status ones (based in Summer values)

In [33]:
# STEP 2: INITIALISE TRAJECTORY GROUP
# Group trajectories and then cycle through a group to determine distance, vector and rain status.
# check out traj_trajgroup_basics_examle.py

# Initialise Trajectories and Create a Trajectory Group
test=[]
test= py.make_trajectorygroup(r'/Volumes/Seagate_Backup/hysplit4/pysplitgen/test/*summer*')
# Calculate distance and vectors
for traj in test:
    traj.calculate_distance()
    traj.calculate_vector()

In [32]:
rainylist =[]

for traj in test:
    traj.set_rainstatus()
    if traj.rainy:
        rainylist.append(traj)
        
rainy_test=py.TrajectoryGroup(rainylist)

dry_test= test - rainy_test
print ('Total trajectroies in dry_test group:', dry_test.trajcount)
print ('Total trajectroies in rainy_test group:', rainy_test.trajcount)
print ('Total trajectroies in group:', test.trajcount)

Total trajectroies in dry_test group: 44
Total trajectroies in rainy_test group: 46
Total trajectroies in group: 90


#### Calculating the integration error

- first the reversetraj has to be loaded, this is because the integration error is the physical and numerical error. This is calculated by the distance between the original trajectory and the reverse trajectory start and end points. 

- then the integration error can be calculated by traj.calculate_integrationerr()

    - this also should inclue te traj.integration_error (the relative error%) and traj.integration_error_abs (the absolute error in meters)

In [40]:
# integration error filtering# integr 
#first load reverse Traj
for traj in test:
    traj.load_reversetraj()
#then calculate integration error
# Values computed when calling ``Trajectory.calculate_integrationerr()``:
#     ``Trajectory.integration_error``, the relative error (%)
#     ``Trajectory.integration_error_abs``, the absolute error (meters)


Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070101041.0 has a bad reverse trajectory: 
	
                          168 hours instead of 4 hours
Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070102041.0 has a bad reverse trajectory: 
	
                          168 hours instead of 28 hours
Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070103041.0 has a bad reverse trajectory: 
	
                          168 hours instead of 52 hours
Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070104041.0 has a bad reverse trajectory: 
	
                          168 hours instead of 76 hours
Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070105041.0 has a bad reverse trajectory: 
	
                          168 hours instead of 100 hours
Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070106041.0 has a bad

In [42]:
c=0
for traj in test:
    traj.calculate_integrationerr()
    print(c)
    c+=1
    




Integration error calculation skipped for
                  Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070101041.0
0
Integration error calculation skipped for
                  Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070102041.0
1
Integration error calculation skipped for
                  Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070103041.0
2
Integration error calculation skipped for
                  Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070104041.0
3
Integration error calculation skipped for
                  Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070105041.0
4
Integration error calculation skipped for
                  Trajectory /Volumes/Seagate_Backup/hysplit4/pysplitgen/test/testjan1100summer20070106041.0
5
Integration error calculation skipped for
                  Trajectory /Volumes/Se

In [45]:
test[10].integration_error

0.010168053088224056

The problem with the code below (relative errors and the integration_error code) is that for the trajectories which do not calculate integration erros it still tries to loop through them. 

- solution would be to try and create a trajectory group which excludes the trajectories which do not fit in, and do not have trajectories integration errors.



- how to do this???

In [38]:
relative_errors = [traj.integration_error for traj in test]
cutoff = np.mean(relative_errors) + (np.std(relative_errors) * 2)

print('Integration error upper limit: ', cutoff)

AttributeError: 'Trajectory' object has no attribute 'integration_error'

NameError: name 'python' is not defined